# 5. Código
********************
********************

In [19]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.spatial import distance

In [ ]:
def get_distances(S,p):
    dist=distance.cdist(S,np.array(p).reshape((1,2)))
    return sum(dist)

def get_p(theta):
    return (np.cos(theta)*1000,np.sin(theta)*1000)

In [ ]:
def solve_modelo(dist_s1,p2_max_dist,N=20,plot=False):
    """
    Función principal para la resolución del problema.
    Soluciona la minimización de suma de distancias de dos conjuntos de puntos inscritos en la circunferencia de centro 0
    y radio 1000 a dos puntos de entrega.
    
    Parametros:
        dist_s1: Int. Distribución inicial de puntos del conjunto S1. Toma valores en {1,2,3}
            1: Distribución de puntos (i,j) con i = −500, . . . , 500, j = −500, . . . , 500.
            2: Distribucion de puntos (i,j) con i = 0, . . . , 500, j = −500, . . . , i.
            3: Distribución de puntos seleccionados aleatoriamente descritos en el problema.
            
        p2_max_dist: Bool. True si el segundo punto de entrega se encuentra diametralmente opuesto a p1.
                        Si es False, este segundo punto es tambiénparte de la optimización.
        
        N: Int. Cantidad de puntos iniciales distintos a ocupar. Son elegidos aleatoriamente según una dist uniforme a lo largo de la circunferencia.
        
        plot: Bool.
    """
    S = [((i*100),(j*100)) for i in range(-10,10+1) for j in range(-10,10+1) if (i*100)**2+(j*100)**2<=(1000)**2]
    if dist_s1==1:
        S1 = [((i*100),(j*100)) for i in range(-5,5+1) for j in range(-5,5+1)]
    elif dist_s1==2:
        S1 = [((i*100),(j*100)) for i in range(0,5+1) for j in range(-5,5+1) if j<=i]
    elif dist_s1==3:
        S1= [(0, 0), (0, 100), (0, 300), (0, 500), (100, 200), (100, 300),(200,-200), (-300, 300), (1000, 0), (-300,-400)]
    else:
        raise ValueError('Valor dist_s1 no reconocido. Los valores aceptados son {1,2,3}.')
        
    S2 = list(set(S).difference(set(S1)))
    
    F = lambda theta: get_distances(S1,get_p(theta[0])) + get_distances(S2,get_p(theta[1]))
    
    if p2_max_dist:    
        f = lambda theta: F([theta,theta+np.pi])

        lb = [0]
        ub = [2*np.pi]
        cotas = list(zip(lb,ub))
        
        best_x=[0,0]
        best_val=np.infty
        
        for i in range(N):
            p0 = np.random.rand(1)*2*np.pi

            m = minimize(f,p0, options={'maxiter':1000}, bounds = cotas)
            if m.fun < best_val:
                best_x=m.x
                best_val=m.fun
        p = [get_p(best_x),get_p(best_x+np.pi)]
    else:
        f = lambda theta: F(theta)

        lb = [0,0]
        ub = [2*np.pi,2*np.pi]
        cotas = list(zip(lb,ub))
        
        best_x=[0,0]
        best_val=np.infty
        
        for i in range(N):
            p0 = np.random.rand(2)*2*np.pi
        
            m = minimize(f,p0, options={'maxiter':1000}, bounds = cotas)
            if m.fun < best_val:
                best_x=m.x
                best_val=m.fun
        p = [get_p(best_x[0]),get_p(best_x[1])]
        
    if plot:
        p_label=['p1','p2']
        plt.scatter(list(zip(*S2))[0],list(zip(*S2))[1],c='b',marker='.',label='S2')
        plt.scatter(list(zip(*S1))[0],list(zip(*S1))[1],c='r',marker='.',label='S1')
        plt.scatter(list(zip(*p))[0],list(zip(*p))[1],c='g',marker='o',alpha=0.7)
        plt.scatter(list(zip(*p))[0],list(zip(*p))[1],c='g',marker='o',alpha=0.7)
        plt.legend()
        for i, txt in enumerate(p_label):
            plt.annotate(txt, p[i], xytext=[a+20 for a in p[i]])
            plt.savefig('sol_dist_{}{}.pdf'.format(dist_s1,'_maxdist'*p2_max_dist),bbox_inches='tight')
    return p,m

In [ ]:
p,m=solve_modelo(1,True,plot=True)